# Loading data

In [ ]:
# import packages
import pickle
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import time
from datetime import datetime
import plotly.io as pio
from plotly.subplots import make_subplots
import os
from sklearn.neighbors import KernelDensity

## 3DMM

### Original model including everything

In [ ]:
# load model
models_folder = './3DMM/UHM_models/'
currnet_model = 'UHM' 

# UHM model with all the components fused together (i.e. ears, inner mouth, and teeth)
model_name = 'head_model_global_align'

model_file = open(models_folder + model_name + '.pkl', 'rb')
model_dict = pickle.load(model_file)
model_file.close()

In [ ]:
# turning coordinates system to be in millimeter units
scale_factor = 100

# get model parameteres
mean_shape = scale_factor*model_dict['Mean']
mean_shape_CCS = mean_shape.reshape(-1,3)
eigen_vec = model_dict['Eigenvectors']
eigen_vec_num =  model_dict['Eigenvectors'].shape[1]
eigen_val = model_dict['EigenValues']
trilist = model_dict['Trilist']
vertices_num = model_dict['Number_of_vertices']

In [ ]:
# load modules (landmarks and masks)
modules_folder = models_folder + '/Landmarks and masks/'

modules_to_load = ['68_land_idxs'] # EEG_10_20_full_model / '49_plus_ears_land_idxs' / '68_land_idxs'

landmarks = []
landmarks_names = []
landmarks_groups = []

for currnet_module_name in modules_to_load:
    module_file = open(modules_folder + currnet_module_name + '.pkl', 'rb')
    currnet_module = pickle.load(module_file)
    module_file.close()
    if currnet_module_name=='EEG_10_20':
        currnet_module_names = list(currnet_module.keys())
        currnet_module = np.asarray(list(currnet_module.values()))
    else:
        currnet_module_names = list(map(str, 1+np.arange(len(currnet_module))))
        
    landmarks.append(currnet_module)
    landmarks_names.append(currnet_module_names)
    landmarks_groups.append(np.arange(len(currnet_module)))

# turn list of lists into one list
landmarks = [item for items in landmarks for item in items]
landmarks_names = [item for items in landmarks_names for item in items]

num_of_landmarks = len(landmarks)

### Lighter model including everything but eyes, teeth and inner mouth cavity

In [ ]:
# load model
light_models_folder = './3DMM/UHM_models/'
light_currnet_model = 'UHM' 

# UHM model with all the components fused together (i.e. ears, inner mouth, and teeth)
light_model_name = 'head_model_global_align_no_mouth_and_eyes'

light_model_file = open(light_models_folder + light_model_name + '.pkl', 'rb')
light_model_dict = pickle.load(light_model_file)
light_model_file.close()

In [ ]:
# turning cartesian coordinates system to be in millimeter units
light_scale_factor = 100

# get model parameteres
light_mean_shape = light_scale_factor*light_model_dict['Mean']
light_mean_shape_CCS = light_mean_shape.reshape(-1,3)
light_eigen_vec = light_model_dict['Eigenvectors']
light_eigen_vec_num =  light_model_dict['Eigenvectors'].shape[1]
light_eigen_val = light_model_dict['EigenValues']
light_trilist = light_model_dict['Trilist']
light_vertices_num = light_model_dict['Number_of_vertices']

In [ ]:
# load modules (landmarks and masks)
modules_folder = models_folder + '/Landmarks and masks/'

modules_to_load = ['EEG_10_20'] # EEG_10_20 / '49_plus_ears_land_idxs' / '68_land_idxs'

light_landmarks = []
light_landmarks_names = []
light_landmarks_groups = []

for currnet_module_name in modules_to_load:
    module_file = open(modules_folder + currnet_module_name + '.pkl', 'rb')
    currnet_module = pickle.load(module_file)
    module_file.close()
    if currnet_module_name=='EEG_10_20':
        currnet_module_names = list(currnet_module.keys())
        currnet_module = np.asarray(list(currnet_module.values()))
    else:
        currnet_module_names = list(map(str, 1+np.arange(len(currnet_module))))
        
    light_landmarks.append(currnet_module)
    light_landmarks_names.append(currnet_module_names)
    light_landmarks_groups.append(np.arange(len(currnet_module)))

# turn list of lists into one list
light_landmarks = [item for items in light_landmarks for item in items]
light_landmarks_names = [item for items in light_landmarks_names for item in items]

num_of_light_landmarks = len(light_landmarks)

### Matching landmark indices between models

In [ ]:
light_facial_landmarks = landmarks

for current_landmark_index, current_landmark_vertex in enumerate(landmarks):
    original_model_coordinates = mean_shape_CCS[current_landmark_vertex]
    new_model_vertex_diffs = np.linalg.norm(light_mean_shape_CCS-original_model_coordinates, axis=1)
    light_facial_landmarks[current_landmark_index] = np.argmin(new_model_vertex_diffs)

### Model Choosing

In [ ]:
choose_light_model = True

In [ ]:
if choose_light_model:
    landmarks = np.concatenate((light_landmarks, light_facial_landmarks))
    landmarks_names = list(np.concatenate((light_landmarks_names, landmarks_names)))
    mean_shape = light_mean_shape
    mean_shape_CCS = light_mean_shape_CCS
    eigen_vec = light_eigen_vec
    eigen_vec_num =  light_eigen_vec_num
    eigen_val = light_eigen_val
    trilist = light_trilist
    vertices_num = light_vertices_num

# Definitions

In [ ]:
distinct_landmarks_names = np.array([37, 40, 43, 46, 49, 55, 31, 9])
rigid_facial_landmarks_names = np.array([37, 40, 43, 46, 28, 1, 17])

center_of_the_eyebrows = np.array([20, 25])
corners_of_the_eyebrows = np.array([18, 22, 23, 27])
corners_of_the_eyes = np.array([37, 40, 43, 46])
sides_of_the_face = np.array([1, 17])
nose_bone = np.array([28, 31])
lower_nose = np.array([32, 34, 36])
corners_of_the_mouth = np.array([49, 55])
chin = np.array([9])

facial_landmarks = np.concatenate((center_of_the_eyebrows, corners_of_the_eyebrows, corners_of_the_eyes, sides_of_the_face, nose_bone, lower_nose))
                                   #,corners_of_the_mouth, chin))
selected_facial_indices = np.sort(facial_landmarks+num_of_light_landmarks-1)

selected_EEG_10_20_landmark_names = light_landmarks_names
selected_EEG_10_20_indices = []
for current_index, current_landmark_name in enumerate(selected_EEG_10_20_landmark_names):
    selected_EEG_10_20_indices.append(landmarks_names.index(current_landmark_name))
selected_EEG_10_20_indices = np.asarray(selected_EEG_10_20_indices)

In [ ]:
MRI_facial_landmarks_names = ['1', '2', '3', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30',
                        '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48']
MRI_facial_landmarks = [int(current_landmark)+20 for current_landmark in MRI_facial_landmarks_names]

In [ ]:
selected_indices = np.concatenate((MRI_facial_landmarks, selected_EEG_10_20_indices))
selected_indices_names = np.take(landmarks_names, selected_indices)

In [ ]:
feature_sets = {}
feature_sets['eye corners & eyebrow corners'] = np.concatenate((corners_of_the_eyebrows, corners_of_the_eyes))
feature_sets['eye corners & eyebrow centers'] = np.concatenate((center_of_the_eyebrows, corners_of_the_eyes))
feature_sets['eye corners & eyebrow corners and center'] = np.concatenate((center_of_the_eyebrows, corners_of_the_eyebrows, corners_of_the_eyes))
feature_sets['eye corners & nose bone'] = np.concatenate((corners_of_the_eyes, nose_bone))
feature_sets['nose bone & lower nose'] = np.concatenate((nose_bone, lower_nose))
feature_sets['MRI_facial_landmarks'] = np.array(MRI_facial_landmarks)

for current_key in feature_sets:
    feature_sets[current_key] = feature_sets[current_key]+num_of_light_landmarks-1
    feature_sets[current_key] = list(map(str, feature_sets[current_key]))

# IXI

## Definitions

In [ ]:
registration_scale_factor = 0.001

In [ ]:
datasets_folder = './MRI_datasets/'
current_dataset_name = 'IXI'
dataset_filename = 'Dataset.xlsx'

In [ ]:
current_subject_dataframe = pd.ExcelFile(datasets_folder+current_dataset_name+'/'+dataset_filename)
current_sheet_names = current_subject_dataframe.sheet_names
current_num_of_sheets = len(current_sheet_names)

skin_coordinates_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin coordinates')
skin_normals_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin normals')
skin_geodesic_distances_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin distances')
inverse_matrices_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Inverse transformations')
stats_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Stats')

In [ ]:
skin_coordinates_df = pd.read_excel(datasets_folder+current_dataset_name+'/'+dataset_filename, sheet_name=skin_coordinates_index, index_col=0)
skin_geodesic_distances_df = pd.read_excel(datasets_folder+current_dataset_name+'/'+dataset_filename, sheet_name=skin_geodesic_distances_index, index_col=0)
stats_df = pd.read_excel(datasets_folder+current_dataset_name+'/'+dataset_filename, sheet_name=stats_index, index_col=0)

In [ ]:
skin_coordinates_columns_names = list(skin_coordinates_df.columns)
only_coordinates_columns_indices = []

for i in range(len(skin_coordinates_columns_names)):
    if 'indices' not in skin_coordinates_columns_names[i]:
        only_coordinates_columns_indices.append(i)

In [ ]:
if 1:
    max_euclidean_distance = 75e-3

    relevant_indices = []
    for desired_landmark_index, desired_landmark_name in enumerate(skin_coordinates_df.index[:num_of_light_landmarks]):
        desired_landmark_data = skin_coordinates_df.loc[desired_landmark_name, :]
        desired_landmark_subjects_coordinates = desired_landmark_data.iloc[np.array(only_coordinates_columns_indices)]
        desired_landmark_subjects_coordinates = np.array(desired_landmark_subjects_coordinates).reshape(-1, 3)
        if np.where(np.isnan(desired_landmark_subjects_coordinates)==True)[0].size>0:
            valid_coordinates_rows = np.unique(np.where(np.isnan(desired_landmark_subjects_coordinates)==False)[0])
        else:
            valid_coordinates_rows = np.arange(desired_landmark_subjects_coordinates.shape[0])

        valid_rows = valid_coordinates_rows

        desired_landmark_coordinates_mean = np.mean(desired_landmark_subjects_coordinates[valid_rows, :], axis=0)
        euclidean_distances = np.linalg.norm(desired_landmark_subjects_coordinates[valid_rows, :]-desired_landmark_coordinates_mean, axis=1)
        desired_landmark_relevant_indices = np.where(euclidean_distances<max_euclidean_distance)[0]
        relevant_indices.append(desired_landmark_relevant_indices)

    only_valid_score_subjects_rows = relevant_indices[0]
    for desired_landmark_index, desired_landmark_name in enumerate(skin_coordinates_df.index[:num_of_light_landmarks]):
        only_valid_score_subjects_rows = np.intersect1d(relevant_indices[desired_landmark_index], only_valid_score_subjects_rows)
else:
    score_ratio_threshold = 1
    only_valid_score_subjects_rows = np.sort(np.argsort(stats_df.loc['unique_correspondence_final_loss', :].values)[:int(score_ratio_threshold*stats_df.shape[1])])

# ADNI

## Definitions

In [ ]:
registration_scale_factor = 0.001

In [ ]:
datasets_folder = './MRI_datasets/'
current_dataset_name = 'ADNI'
dataset_filename = 'Dataset.xlsx'

In [ ]:
current_subject_dataframe = pd.ExcelFile(datasets_folder+current_dataset_name+'/'+dataset_filename)
current_sheet_names = current_subject_dataframe.sheet_names
current_num_of_sheets = len(current_sheet_names)

skin_coordinates_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin coordinates')
skin_normals_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin normals')
skin_geodesic_distances_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Skin distances')
inverse_matrices_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Inverse transformations')
stats_index = next(i for i in range(len(current_sheet_names)) if current_sheet_names[i]=='Stats')

In [ ]:
skin_coordinates_df = pd.read_excel(datasets_folder+current_dataset_name+'/'+dataset_filename, sheet_name=skin_coordinates_index, index_col=0)
skin_geodesic_distances_df = pd.read_excel(datasets_folder+current_dataset_name+'/'+dataset_filename, sheet_name=skin_geodesic_distances_index, index_col=0)
stats_df = pd.read_excel(datasets_folder+current_dataset_name+'/'+dataset_filename, sheet_name=stats_index, index_col=0)

In [ ]:
skin_coordinates_columns_names = list(skin_coordinates_df.columns)
only_coordinates_columns_indices = []

for i in range(len(skin_coordinates_columns_names)):
    if 'indices' not in skin_coordinates_columns_names[i]:
        only_coordinates_columns_indices.append(i)

In [ ]:
if 1:
    max_euclidean_distance = 75e-3

    relevant_indices = []
    for desired_landmark_index, desired_landmark_name in enumerate(skin_coordinates_df.index[:num_of_light_landmarks]):
        desired_landmark_data = skin_coordinates_df.loc[desired_landmark_name, :]
        desired_landmark_subjects_coordinates = desired_landmark_data.iloc[np.array(only_coordinates_columns_indices)]
        desired_landmark_subjects_coordinates = np.array(desired_landmark_subjects_coordinates).reshape(-1, 3)
        if np.where(np.isnan(desired_landmark_subjects_coordinates)==True)[0].size>0:
            valid_coordinates_rows = np.unique(np.where(np.isnan(desired_landmark_subjects_coordinates)==False)[0])
        else:
            valid_coordinates_rows = np.arange(desired_landmark_subjects_coordinates.shape[0])

        valid_rows = valid_coordinates_rows

        desired_landmark_coordinates_mean = np.mean(desired_landmark_subjects_coordinates[valid_rows, :], axis=0)
        euclidean_distances = np.linalg.norm(desired_landmark_subjects_coordinates[valid_rows, :]-desired_landmark_coordinates_mean, axis=1)
        desired_landmark_relevant_indices = np.where(euclidean_distances<max_euclidean_distance)[0]
        relevant_indices.append(desired_landmark_relevant_indices)

    only_valid_score_subjects_rows = relevant_indices[0]
    for desired_landmark_index, desired_landmark_name in enumerate(skin_coordinates_df.index[:num_of_light_landmarks]):
        only_valid_score_subjects_rows = np.intersect1d(relevant_indices[desired_landmark_index], only_valid_score_subjects_rows)
else:
    score_ratio_threshold = 1
    only_valid_score_subjects_rows = np.sort(np.argsort(stats_df.loc['unique_correspondence_final_loss', :].values)[:int(score_ratio_threshold*stats_df.shape[1])])

# Mathematical analysis

In [ ]:
skin_relevant_coordinates_df = skin_coordinates_df.iloc[:, np.array(only_coordinates_columns_indices)].T.iloc[:, np.concatenate((selected_EEG_10_20_indices, np.arange(21,skin_coordinates_df.shape[0]-2)))]

In [ ]:
skin_coordinates_reshaped = np.zeros((int(skin_relevant_coordinates_df.shape[0]/3), skin_relevant_coordinates_df.shape[1]*3))

In [ ]:
for i in range(skin_coordinates_reshaped.shape[0]):
    skin_coordinates_reshaped[i, :] = skin_relevant_coordinates_df.values.reshape(int(skin_relevant_coordinates_df.shape[0]/3), skin_relevant_coordinates_df.shape[1]*3)[i, :].reshape(-1, skin_relevant_coordinates_df.shape[1]).T.ravel()

In [ ]:
multiindex_up_names = list(np.repeat(selected_indices_names, 3))
multiindex_down_names = ['x', 'y', 'z']*len(multiindex_up_names)

In [ ]:
skin_relevant_coordinates_df = pd.DataFrame(data=skin_coordinates_reshaped, 
                                            columns=pd.MultiIndex.from_tuples(zip(multiindex_up_names, multiindex_down_names)))

## Kernel density estimation

In [ ]:
def kernel_based_empirical_risk(X, h):
    # Define shortcuts for K K2 and K_ast
    K = lambda u: 1/np.sqrt(2*np.pi) * np.exp(-u**2 / 2)  # Define the kernel N(0,1)
    K2 = lambda u: 1/np.sqrt(2*np.pi * 2) * np.exp(-u**2 / (2 * 2))  # Define the kernel with sigma^2=2
    K_ast = lambda u: K2(u) - 2*K(u)    
    n = len(X)
    J = 2/(n*h) * K(0)  #  Left term [1] pag. 316, Eq (20.25)
    for X_i in X:
        for X_j in X:
            J += 1 / (h * n**2) * K_ast(np.linalg.norm(X_i-X_j)/h)
    return J

In [ ]:
joint_axis_grid_size = 4
marginal_axis_grid_size = int(1.5*joint_axis_grid_size)
if (marginal_axis_grid_size%2)==1:
    marginal_axis_grid_size+=1

surface_count = 32

bw_risk_estimation_samples_percentage=0.1
bw_risk_estimation_indices = np.sort(np.random.choice(skin_relevant_coordinates_df.values.shape[0],
                                 int(skin_relevant_coordinates_df.values.shape[0]*bw_risk_estimation_samples_percentage),
                                                   replace=False))
bw_options = np.linspace(2.5, 7.5, 11)/1000

In [ ]:
all_selected_indices = np.concatenate((selected_EEG_10_20_indices, np.array(MRI_facial_landmarks)))
all_selected_indices_names = np.take(landmarks_names, all_selected_indices)

### Probability distribution function

In [ ]:
coordinates_list = []

if 0: # for PMF visualizations
    distribution_function_values_mat = np.zeros((len(all_selected_indices_names), marginal_axis_grid_size**3))
    axis_diff = marginal_axis_grid_size-joint_axis_grid_size
    linspace_size = marginal_axis_grid_size
else: # for joint PMF and MI values
    distribution_function_values_mat = np.zeros((len(all_selected_indices_names), joint_axis_grid_size**3))
    axis_diff = 0
    linspace_size = joint_axis_grid_size
    
#for desired_landmark_index, desired_landmark_name in enumerate(landmarks_names[:1]):
for desired_landmark_index, desired_landmark_name in enumerate(all_selected_indices_names):
    print(("Started " + desired_landmark_name))
    current_samples = skin_relevant_coordinates_df.loc[:, desired_landmark_name]
    current_samples = current_samples[~current_samples.isnull().any(axis=1)]
    current_samples = current_samples.values[np.where(current_samples.values.all(axis=1))[0], :]
    current_risk_estimation_samples = skin_relevant_coordinates_df.loc[bw_risk_estimation_indices, desired_landmark_name].values

    current_J = [kernel_based_empirical_risk(current_risk_estimation_samples, current_bw) for current_bw in bw_options]
    selected_bw = bw_options[np.argmin(current_J)]
    print(selected_bw)

    current_kde = KernelDensity(kernel="gaussian", bandwidth=selected_bw)
    current_kde.fit(current_samples)
    
    x_coordinates = current_samples[:, 0]
    x_coordinates_mean = np.mean(x_coordinates)
    x_coordinates_std = np.std(x_coordinates)
    x_min = x_coordinates_mean-3*x_coordinates_std
    x_max = x_coordinates_mean+3*x_coordinates_std
    x_bin_width = (x_max-x_min)/joint_axis_grid_size
    x_grid = np.linspace(x_min+0.5*x_bin_width-(0.5*axis_diff*x_bin_width), x_max-0.5*x_bin_width+(0.5*axis_diff*x_bin_width), linspace_size)

    y_coordinates = current_samples[:, 1]
    y_coordinates_mean = np.mean(y_coordinates)
    y_coordinates_std = np.std(y_coordinates)
    y_min = y_coordinates_mean-3*y_coordinates_std
    y_max = y_coordinates_mean+3*y_coordinates_std
    y_bin_width = (y_max-y_min)/joint_axis_grid_size
    y_grid = np.linspace(y_min+0.5*y_bin_width-(0.5*axis_diff*y_bin_width), y_max-0.5*y_bin_width+(0.5*axis_diff*y_bin_width), linspace_size)

    z_coordinates = current_samples[:, 2]
    z_coordinates_mean = np.mean(z_coordinates)
    z_coordinates_std = np.std(z_coordinates)
    z_min = z_coordinates_mean-3*z_coordinates_std
    z_max = z_coordinates_mean+3*z_coordinates_std
    z_bin_width = (z_max-z_min)/joint_axis_grid_size
    z_grid = np.linspace(z_min+0.5*z_bin_width-(0.5*axis_diff*z_bin_width), z_max-0.5*z_bin_width+(0.5*axis_diff*z_bin_width), linspace_size)
    
    X, Y, Z = np.meshgrid(x_grid, y_grid, z_grid)
    coordinates_list.append([X, Y, Z])
    xyz = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
    
    distribution_function_values = current_kde.score_samples(xyz)
    distribution_function_values = np.exp(distribution_function_values)
    distribution_function_values = distribution_function_values/np.sum(distribution_function_values)
    
    distribution_function_values_mat[desired_landmark_index, :] = distribution_function_values
    
    if 0:
        fig = go.Figure(data=go.Volume(
            x=X.flatten(),
            y=Y.flatten(),
            z=Z.flatten(),
            value=distribution_function_values,
            opacity=1,
            surface_count=surface_count,
            ))
        fig.update_layout(title_text=(desired_landmark_name+' distribution function'+"\ndataset: "+dataset_filename), title_x=0.5)
        fig.show()

In [ ]:
# plot 3d scatter
desired_landmark_plot = '3'
for desired_landmark_index, desired_landmark_name in enumerate(all_selected_indices_names):
    if desired_landmark_name!=desired_landmark_plot:
        continue
    current_samples = skin_relevant_coordinates_df.loc[:, desired_landmark_name]
    current_samples = current_samples[~current_samples.isnull().any(axis=1)]
    current_samples = current_samples.values[np.where(current_samples.values.all(axis=1))[0], :]


fig = px.scatter_3d(x=current_samples[:, 0],
                    y=current_samples[:, 1],
                    z=current_samples[:, 2]
                    )
fig.show()

In [ ]:
if 1:
    array_folder = "./3DMM/Statistical_analysis/"
    array_filename = (current_dataset_name+"_PMF_"+str(joint_axis_grid_size))
    array_filetype = '.npy'

    array_path = array_folder + array_filename + array_filetype
    
    if 1:
        with open(array_path, 'wb') as file:
            np.save(file, distribution_function_values_mat)
    else:
        with open(array_path, 'rb') as file:
            distribution_function_values_mat = np.load(file)

In [ ]:
current_index=2    
X, Z, Y = coordinates_list[current_index]

current_landmark_distribution_function_values = distribution_function_values_mat[current_index, :]

fig=go.Figure()

fig.add_trace(
    go.Volume(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=current_landmark_distribution_function_values,
        opacity=0.9,
        isomin=0.5e-3,
        surface_count=surface_count,
        colorbar=dict(lenmode='pixels', len=1000, thickness=50, tickfont=dict(size=30), x=1.01),
        colorscale='YlGnBu'
    )
)

fig.show()

In [ ]:
if 1:
    fig.update_layout(
        title_text=("Landmarks distribution function<br><sup>"+current_dataset_name+"</sup>"),
        title_x=0.5,
        scene_camera=scene_camera[0],
        scene=dict(
               #annotations=annotation_list,
               #xaxis_visible=False, yaxis_visible=False, zaxis_visible=False
        )
    )
    fig.show()
else:
    fig.update_layout(
        title_text=("Landmarks distribution function<br><sup>"+current_dataset_name+"</sup>"),
        title_x=0.5,
        scene=dict(
               annotations=annotation_list,
               #xaxis_visible=False, yaxis_visible=False, zaxis_visible=False
        )
    )
    
    fig.layout.height = 2000
    fig.layout.width = fig.layout.height
    fig.layout.margin.t = 0
    fig.layout.margin.b = 0
    fig.layout.margin.l = 0
    fig.layout.margin.r = 0

    fig.update_layout(title_text="")

    timestamp_string = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    timestamp_string = timestamp_string.replace('_2022_', '_22_')

    folder = "./Cranium_estimation_paper/Figures/PMF/"
    filetype = ".jpg"
    
    for current_index in range(len(scene_camera[:1])):
        fig.update_layout(scene_camera=scene_camera[current_index])
        
        filename = "figure_PMF_"+str(current_index)+"_"+str(joint_axis_grid_size)+"_"+str(marginal_axis_grid_size)+"_"+str(isomin_starting_percentile)+"_"+timestamp_string
        path = folder + filename + filetype

        pio.write_image(fig, path, scale=1)   

### Joint probability distribution function

In [ ]:
#MI_mat = np.zeros((len(all_selected_indices_names), len(all_selected_indices_names)))
MI_mat = np.zeros((len(all_selected_indices_names[len(light_landmarks):]), len(all_selected_indices_names[:len(light_landmarks)])))
load_MI_mat = False

if load_MI_mat:
    array_folder = "./3DMM/Head_instances/"
    array_filetype = '.npy'
    
    last_loaded_index = 0
    
    for current_landmark_index in range(len(all_selected_indices_names)):
        current_index_name = all_selected_indices_names[current_landmark_index]
        array_filename = ("MI_"+str(joint_axis_grid_size)+"_"+str(current_index_name))
        array_path = array_folder + array_filename + array_filetype
        
        exists = os.path.exists(array_path)
        
        if not exists:
            break
        else:
            with open(array_path, 'rb') as file:
                current_landmark_MI_vals = np.load(file)
            MI_mat[current_landmark_index, :] = current_landmark_MI_vals
            last_loaded_index = current_landmark_index
            
    print(last_loaded_index)

In [ ]:
for desired_landmark_index_i, desired_landmark_name_i in enumerate(all_selected_indices_names[len(light_landmarks):]):
    print(("Started " + desired_landmark_name_i))
    if load_MI_mat:
        if last_loaded_index>=desired_landmark_index_i:
            continue       
        
    current_samples_i = skin_relevant_coordinates_df.loc[:, desired_landmark_name_i]
    no_nan_rows_i = np.where(~current_samples_i.isnull().any(axis=1).values==True)[0]
    no_zeros_rows_i = np.where(current_samples_i.values.all(axis=1))[0]
    current_risk_estimation_samples_i = skin_relevant_coordinates_df.loc[bw_risk_estimation_indices, desired_landmark_name_i].values

    for desired_landmark_index_j, desired_landmark_name_j in enumerate(all_selected_indices_names[:len(light_landmarks)]):
        '''if desired_landmark_index_i==desired_landmark_index_j:
            MI_mat[desired_landmark_index_i, desired_landmark_index_j] = 0
        elif desired_landmark_index_i>desired_landmark_index_j:
            MI_mat[desired_landmark_index_i, desired_landmark_index_j] = MI_mat[desired_landmark_index_j, desired_landmark_index_i]
        else:'''
        print(("Started " + desired_landmark_name_i + " and " + desired_landmark_name_j))
        current_samples_j = skin_relevant_coordinates_df.loc[:, desired_landmark_name_j]
        no_nan_rows_j = np.where(~current_samples_j.isnull().any(axis=1).values==True)[0]
        no_zeros_rows_j = np.where(current_samples_j.values.all(axis=1))[0]
        current_risk_estimation_samples_j = skin_relevant_coordinates_df.loc[bw_risk_estimation_indices, desired_landmark_name_j].values

        no_nan_rows_ij = np.intersect1d(no_nan_rows_i, no_nan_rows_j)
        no_zeros_rows_ij = np.intersect1d(no_zeros_rows_i, no_zeros_rows_j)
        only_relevant_rows_ij = np.intersect1d(no_nan_rows_ij, no_zeros_rows_ij)
        current_samples = np.concatenate([current_samples_i.iloc[only_relevant_rows_ij, :], current_samples_j.iloc[only_relevant_rows_ij, :]], axis=1)

        current_risk_estimation_samples = np.concatenate([current_risk_estimation_samples_i, current_risk_estimation_samples_j], axis=1)

        current_J = [kernel_based_empirical_risk(current_risk_estimation_samples, current_bw) for current_bw in bw_options]
        selected_bw = bw_options[np.argmin(current_J)]
        #selected_bw = 3.5
        #print(selected_bw)

        current_kde = KernelDensity(kernel="gaussian", bandwidth=selected_bw)
        current_kde.fit(current_samples)

        x_coordinates_i = current_samples[:, 0]
        x_coordinates_i_mean = np.mean(x_coordinates)
        x_coordinates_i_std = np.std(x_coordinates)
        x_min_i = x_coordinates_i_mean-3*x_coordinates_i_std
        x_max_i = x_coordinates_i_mean+3*x_coordinates_i_std
        x_bin_width_i = (x_max_i-x_min_i)/joint_axis_grid_size
        x_grid_i = np.linspace(x_min_i+0.5*x_bin_width_i, x_max_i-0.5*x_bin_width_i, joint_axis_grid_size)

        y_coordinates_i = current_samples[:, 1]
        y_coordinates_i_mean = np.mean(y_coordinates)
        y_coordinates_i_std = np.std(y_coordinates)
        y_min_i = y_coordinates_i_mean-3*y_coordinates_i_std
        y_max_i = y_coordinates_i_mean+3*y_coordinates_i_std
        y_bin_width_i = (y_max_i-y_min_i)/joint_axis_grid_size
        y_grid_i = np.linspace(y_min_i+0.5*y_bin_width_i, y_max_i-0.5*y_bin_width_i, joint_axis_grid_size)

        z_coordinates_i = current_samples[:, 2]
        z_coordinates_i_mean = np.mean(z_coordinates)
        z_coordinates_i_std = np.std(z_coordinates)
        z_min_i = z_coordinates_i_mean-3*z_coordinates_i_std
        z_max_i = z_coordinates_i_mean+3*z_coordinates_i_std
        z_bin_width_i = (z_max_i-z_min_i)/joint_axis_grid_size
        z_grid_i = np.linspace(z_min_i+0.5*z_bin_width_i, z_max_i-0.5*z_bin_width_i, joint_axis_grid_size)

        x_coordinates_j = current_samples[:, 3]
        x_coordinates_j_mean = np.mean(x_coordinates)
        x_coordinates_j_std = np.std(x_coordinates)
        x_min_j = x_coordinates_j_mean-3*x_coordinates_j_std
        x_max_j = x_coordinates_j_mean+3*x_coordinates_j_std
        x_bin_width_j = (x_max_j-x_min_j)/joint_axis_grid_size
        x_grid_j = np.linspace(x_min_j+0.5*x_bin_width_j, x_max_j-0.5*x_bin_width_j, joint_axis_grid_size)

        y_coordinates_j = current_samples[:, 4]
        y_coordinates_j_mean = np.mean(y_coordinates)
        y_coordinates_j_std = np.std(y_coordinates)
        y_min_j = y_coordinates_j_mean-3*y_coordinates_j_std
        y_max_j = y_coordinates_j_mean+3*y_coordinates_j_std
        y_bin_width_j = (y_max_j-y_min_j)/joint_axis_grid_size
        y_grid_j = np.linspace(y_min_j+0.5*y_bin_width_j, y_max_j-0.5*y_bin_width_j, joint_axis_grid_size)

        z_coordinates_j = current_samples[:, 5]
        z_coordinates_j_mean = np.mean(z_coordinates)
        z_coordinates_j_std = np.std(z_coordinates)
        z_min_j = z_coordinates_j_mean-3*z_coordinates_j_std
        z_max_j = z_coordinates_j_mean+3*z_coordinates_j_std
        z_bin_width_j = (z_max_j-z_min_j)/joint_axis_grid_size
        z_grid_j = np.linspace(z_min_j+0.5*z_bin_width_j, z_max_j-0.5*z_bin_width_j, joint_axis_grid_size)

        X_i, Y_i, Z_i, X_j, Y_j, Z_j = np.meshgrid(x_grid_i, y_grid_i, z_grid_i, x_grid_j, y_grid_j, z_grid_j)
        xyz_i_xyz_j = np.vstack([X_i.ravel(), Y_i.ravel(), Z_i.ravel(), X_j.ravel(), Y_j.ravel(), Z_j.ravel()]).T

        joint_distribution_function_values = current_kde.score_samples(xyz_i_xyz_j)
        joint_distribution_function_values = np.exp(joint_distribution_function_values)
        try:
            joint_distribution_function_values = joint_distribution_function_values/np.sum(joint_distribution_function_values)
        except:
            joint_distribution_function_values = 0

        distribution_function_values_i = distribution_function_values_mat[desired_landmark_index_i, :]
        distribution_function_values_j = distribution_function_values_mat[desired_landmark_index_j, :]

        current_MI = 0

        for i in range(distribution_function_values_i.shape[0]):
            i_distribution_value = distribution_function_values_i[i]
            nan_counter=0
            for j in range(distribution_function_values_j.shape[0]):
                j_distribution_value = distribution_function_values_j[j]
                joint_distribution_value = joint_distribution_function_values[i*distribution_function_values_j.shape[0] + j]
                try:
                    current_MI+=joint_distribution_value*np.log(joint_distribution_value/(i_distribution_value*j_distribution_value))
                except:
                    nan_counter+=1
            if nan_counter>0:
                current_MI = current_MI*(distribution_function_values_j.shape[0]/(distribution_function_values_j.shape[0]-nan_counter))
        MI_mat[desired_landmark_index_i, desired_landmark_index_j] = current_MI

In [ ]:
if 1:
    array_folder = "./3DMM/Statistical_analysis/"
    array_filename = (current_dataset_name+"_joint_PMF_"+str(joint_axis_grid_size))
    array_filetype = '.npy'

    array_path = array_folder + array_filename + array_filetype

    with open(array_path, 'wb') as file:
        np.save(file, joint_distribution_function_values)

In [ ]:
if 1:
    array_folder = "./3DMM/Statistical_analysis/"
    array_filename = (current_dataset_name+"_MI_mat_"+str(joint_axis_grid_size))
    array_filetype = '.npy'

    array_path = array_folder + array_filename + array_filetype

    with open(array_path, 'wb') as file:
        np.save(file, MI_mat)

In [ ]:
array_folder = "./3DMM/Statistical_analysis/"
array_filename = ("IXI_MI_mat_"+str(4))
array_filetype = '.npy'

array_path = array_folder + array_filename + array_filetype

with open(array_path, 'rb') as file:
    IXI_MI_mat = np.load(file)

In [ ]:
array_folder = "./3DMM/Statistical_analysis/"
array_filename = ("ADNI_MI_mat_"+str(4))
array_filetype = '.npy'

array_path = array_folder + array_filename + array_filetype

with open(array_path, 'rb') as file:
    ADNI_MI_mat = np.load(file)

## Mutual information

In [ ]:
def figure_saver(fig, filename):
    
    pio.kaleido.scope.mathjax = None

    timestamp_string = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    timestamp_string = timestamp_string.replace('_2022_', '_22_')

    figure_folder = "./Cranium_estimation_paper/Figures/Mutual_information/"
    figure_filename = f"{filename}_{timestamp_string}" #???"experiment_"+str(experiment_number)+"_"+timestamp_string+"_"+filename
    figure_filetype = ".eps"

    figure_path = figure_folder + figure_filename + figure_filetype

    fig.layout.title = ""
    
    fig.layout.margin.t = 0
    fig.layout.margin.b = 0
    fig.layout.margin.l = 0
    fig.layout.margin.r = 10

    pio.write_image(fig, figure_path)#, scale=5)
    pio.write_json(fig, file=figure_folder+figure_filename+".json" ,validate=True, pretty=True, remove_uids=False, engine='json')

### Landmark coordinates

In [ ]:
current_filename = f"IXI_MI_{joint_axis_grid_size}"
IXI_MI_fig = px.imshow(
    MI_mat.T,
    x=list(all_selected_indices_names[len(light_landmarks):]),
    y=list(all_selected_indices_names[:len(light_landmarks)]),
    color_continuous_scale=px.colors.sequential.YlGnBu
)

IXI_MI_fig.update_layout(
    title=f"{current_dataset_name} landmark positions mutual information matrix, joint_axis_grid_size={joint_axis_grid_size}",
    title_x=0.5,
)

IXI_MI_fig.show()

In [ ]:
figure_saver(IXI_MI_fig, current_filename)

In [ ]:
current_filename = f"ADNI_MI_{joint_axis_grid_size}"
ADNI_MI_fig = px.imshow(
    MI_mat.T,
    x=list(all_selected_indices_names[len(light_landmarks):]),
    y=list(all_selected_indices_names[:len(light_landmarks)]),
    #height=400,
    color_continuous_scale=px.colors.sequential.YlGnBu
)

ADNI_MI_fig.update_layout(
    title=f"{current_dataset_name} landmark positions mutual information matrix, joint_axis_grid_size={joint_axis_grid_size}",
    title_x=0.5,
)

IXI_MI_fig.show()

In [ ]:
figure_saver(ADNI_MI_fig, current_filename)

In [ ]:
current_filename = f'Unified_MI_{joint_axis_grid_size}'
color_palette = px.colors.qualitative.Dark24 #Light24

fig_unified_MI = make_subplots(rows=2,
                               cols=1,
                               horizontal_spacing=0,
                               vertical_spacing=0.025,
                               shared_xaxes=True,
                               shared_yaxes=True,
                               x_title='Landmark',
                               y_title='Mutual information value',
                               row_titles=['IXI',
                                           'ADNI']
                                )

fig_unified_MI.add_trace(go.Heatmap(
    z=np.flipud(IXI_MI_mat.T),
    x=list(all_selected_indices_names[len(light_landmarks):]),
    y=list(all_selected_indices_names[:len(light_landmarks)])[::-1],
    coloraxis = "coloraxis"
), 1, 1
                        )

fig_unified_MI.add_trace(go.Heatmap(
    z=np.flipud(ADNI_MI_mat.T),
    x=list(all_selected_indices_names[len(light_landmarks):]),
    y=list(all_selected_indices_names[:len(light_landmarks)])[::-1],
    coloraxis = "coloraxis"
), 2, 1
                        )

fig_unified_MI.update_layout(
    height=1000,
    coloraxis = {'colorscale':'YlGnBu'},
    font = dict(size=12, family="Times new roman"),
    xaxis = dict(
        tickmode = 'linear',
    ),

)
                                      
fig_unified_MI.show()

In [ ]:
figure_saver(fig_unified_MI, current_filename)

In [ ]:
if 1:
    figure_folder = "./Cranium_estimation_paper/Figures/Mutual_information/"
    figure_filetype = ".jpg"
    figure_filename = ("full_landmarks_position_"+str(joint_axis_grid_size)+"_"+filename)

    figure_path = figure_folder + figure_filename + figure_filetype

    fig.layout.title = ""
    fig.update_layout(coloraxis_colorbar=dict(x=0.8))
    
    pio.write_image(fig, figure_path, scale=5)

In [ ]:
first_10_20_EEG_landmark_index=np.where(all_selected_indices_names=='Fz')[0]
indices_10_20_EEG = np.arange(first_10_20_EEG_landmark_index, all_selected_indices_names.shape[0])
indices_face = np.arange(first_10_20_EEG_landmark_index-1)

In [ ]:
MI_mat_selected_rows = np.take(MI_mat, indices_10_20_EEG, axis=0)
MI_mat_selected_rows_cols = np.take(MI_mat_selected_rows, indices_face, axis=1)

In [ ]:
fig = px.imshow(MI_mat_selected_rows_cols , x=all_selected_indices_names[indices_face], y=all_selected_indices_names[indices_10_20_EEG], height=400, color_continuous_scale=px.colors.sequential.YlGnBu)
fig.update_layout(title="Landmarks mutual information matrix<br><sup>"+fixed_filename+"</sup>", title_x=0.5, font=dict(size=7))

fig.show()

In [ ]:
if 1:
    figure_folder = "./Cranium_estimation_paper/Figures/Mutual_information/"
    figure_filetype = ".jpg"
    figure_filename = ("selected_landmarks_position_"+str(joint_axis_grid_size)+"_"+filename)

    figure_path = figure_folder + figure_filename + figure_filetype

    fig.layout.title = ""
    fig.update_layout(coloraxis_colorbar=dict(x=1.01))
    
    pio.write_image(fig, figure_path, scale=5)